In [2]:
from bs4 import BeautifulSoup
import requests
import re
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from datetime import datetime, timedelta
import time
import os
import pandas as pd

In [4]:
all_news_links = []
    
url = 'https://www.firstpost.com/search?q=bangladesh#gsc.tab=0&gsc.q=bangladesh&gsc.sort=date'


base_url = "https://www.firstpost.com/"
country = 'bangladesh'
initial_url = f'{base_url}search?q={country}#gsc.tab=0&gsc.q=bangladesh&gsc.sort=date'

# chrome_options = Options()
# chrome_options.add_argument("--headless")
# driver = webdriver.Chrome(options=chrome_options)
driver = webdriver.Chrome()
driver.get(initial_url)

#this Website displays news articles with the help of Google/enchanced by Google.
#So, each page needs to be separately selected and visited to scrape the links. 
#that can't be done by changing the Page Number in the URl. So, it's done using ActionsChains 'Click' function.

#the links are then filtered to remove unnecessary news, since the search consists of any and all news that 
#remotely mention 'bangaldesh', even if it isn't strongly related to Bangladesh. 
#The news that only shows Scorecard of Cricket Matches are also filtered out. 

page =1 
# for page in range(1,4):

while True:
    
    try:
        buttons = driver.find_elements(By.CLASS_NAME, 'gsc-cursor-page')
        for button in buttons: 
        
            
            if button.get_attribute('innerHTML') == f'{page}':

                ActionChains(driver)\
                .click(button)\
                .perform()
                time.sleep(10)                  

                main_div = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[2]/div/div/div/div/div/div/div[5]/div[2]/div[2]/div/div[1]')


                if main_div:
                    news = main_div.find_elements(By.CSS_SELECTOR, 'a')

                    for n in news:
                        news_text = n.get_attribute('data-ctorig')

                        if news_text in all_news_links:
                            continue
                        else:
                            if 'bangladesh' in news_text or 'bangladeshs' in news_text:
                                if 'cricket-live-score' in news_text:
                                    continue
                                else:
                                    all_news_links.append(news_text)
                            else:
                                continue     
    except:
       
            pass
    
    page+=1
    
    if page == 4:
        
        break

driver.quit()

In [5]:
len(all_news_links)

0

In [3]:
all_news_links = []
    
url = 'https://www.firstpost.com/search?q=bangladesh#gsc.tab=0&gsc.q=bangladesh&gsc.sort=date'


base_url = "https://www.firstpost.com/"
country = 'bangladesh'
initial_url = f'{base_url}search?q={country}#gsc.tab=0&gsc.q=bangladesh&gsc.sort=date'

# chrome_options = Options()
# chrome_options.add_argument("--headless")
# driver = webdriver.Chrome(options=chrome_options)
driver = webdriver.Chrome()
driver.get(url)

#this Website displays news articles with the help of Google/enchanced by Google.
#So, each page needs to be separately selected and visited to scrape the links. 
#that can't be done by changing the Page Number in the URl. So, it's done using ActionsChains 'Click' function.

#the links are then filtered to remove unnecessary news, since the search consists of any and all news that 
#remotely mention 'bangaldesh', even if it isn't strongly related to Bangladesh. 
#The news that only shows Scorecard of Cricket Matches are also filtered out. 

for page in range(1,3):
    
    try:
        buttons = driver.find_elements(By.CLASS_NAME, 'gsc-cursor-page')
        for button in buttons: 
        
            
            if button.get_attribute('innerHTML') == f'{page}':

                ActionChains(driver)\
                .click(button)\
                .perform()
                time.sleep(10)                  

                main_div = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[2]/div/div/div/div/div/div/div[5]/div[2]/div[2]/div/div[1]')


                if main_div:
                    news = main_div.find_elements(By.CSS_SELECTOR, 'a')

                    for n in news:
                        news_text = n.get_attribute('data-ctorig')

                        if news_text in all_news_links:
                            continue
                        else:
                            if 'bangladesh' in news_text or 'bangladeshs' in news_text:
                                if 'cricket-live-score' in news_text:
                                    continue
                                else:
                                    all_news_links.append(news_text)
                            else:
                                continue     
    except:
       
            pass

driver.quit()

counter = 0
data_list = []

for url in all_news_links:
    
    country = 'india'
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)
#     driver = webdriver.Chrome()
    driver.get(url)
    
    try:
        title_tag = driver.find_element(By.CLASS_NAME, 'inner-main-title')
        if title_tag:
            title = title_tag.get_attribute('innerHTML').strip()
        else:
            title = 'Title not found'
            
    except Exception:
        
        title_tag = driver.find_element(By.CLASS_NAME, 'article-title')
        if title_tag:
            title = title_tag.get_attribute('innerHTML').strip()
        else:
            title = 'Title not found'
    
    except:
        
        print(counter)
        title = 'Title not found'

    date_tags = driver.find_elements(By.TAG_NAME, "meta")

    for meta_tag in date_tags:
        if meta_tag.get_attribute("property") == "article:modified_time":
            date_data = meta_tag.get_attribute("content")
            break
        elif meta_tag.get_attribute("property") == "article:published_time":
            date_data = meta_tag.get_attribute("content")
            break

    only_date = date_data.split('T')[0]
    only_time = date_data.split('T')[1]
    time = f"{only_time.split(':')[0]}:{only_time.split(':')[1]}"
    cleaned_date = f"{only_date},{time}"

    source_localtime = datetime.strptime(cleaned_date, "%Y-%m-%d,%H:%M")
    bangladesh_localtime = source_localtime + timedelta(minutes=30)

    p_lists = []
    
    try:
        
        content_div = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[3]/div/div[1]/div[2]/div[4]')
    
        paragraphs = content_div.find_elements(By.CSS_SELECTOR,'p')

        for paras in paragraphs:

            p_lists.append(paras.get_attribute('innerHTML'))
    
    except Exception:
        
        content_div = driver.find_element(By.CLASS_NAME, 'inner-copy')
        
        paragraphs = content_div.find_elements(By.CSS_SELECTOR,'p')

        for paras in paragraphs:

            p_lists.append(paras.get_attribute('innerHTML'))
            
    except:
        
        print(f'content,{counter}')
        p_lists = ['Content Not Found']
        

    full_content = ' '.join(p_lists)


    #This is to remove the news after (With agency inputs) in the content, if present.
    temp_cuts = full_content.split('(With agency inputs)', 1)

    full_content = temp_cuts[0].strip()

    #This is to remove the news after LATEST NEWS in the content if present.
    temp_cuts = full_content.split('Latest News', 1)

    full_content = temp_cuts[0].strip()
    
    #This is to remove the HTML TAGS in the content if present.
    full_content = re.sub('<strong>|</strong>|<em>|</em>','',full_content)

    try:    
        content_summary_tag = driver.find_element(By.CLASS_NAME, 'inner-copy-excerpt')
        content_summary = (content_summary_tag.get_attribute('innerHTML')).strip()
            
    except Exception:
        
        content_summary_tag = driver.find_element(By.CLASS_NAME, 'inner-copy')
        content_summary = (content_summary_tag.get_attribute('innerHTML')).strip()
        
    except:
        
        print(f'summary,{counter}')
        content_summary = 'Content summary not found'
        
    title_translation = 'None'
    summary_translation = 'None'
    content_translation = 'None'
    
#     data_dict = {
#         "country": country,
#         "url": url,
#         "title": title,
#         "timestamp": timestamp,
#         "date": date,
#         "date_str": date_str,
#         "content": full_content,
#         "content_summary": content_summary
#     }
    
    data_dict = {
            "url": url,
            "title": title,
            "content": full_content,
            "content_summary": content_summary,
            "title_translation":title_translation,
            "content_translation":content_translation,
            "summary translation":summary_translation,
            "author": author,
            "country": country,
            'source_localtime': source_localtime,
            'bangladesh_localtime': bangladesh_localtime

        }

    counter+=1


    if (title != "Title not found" and full_content != "Content Not Found" and content_summary != "Content summary not found"):
        if data_dict not in data_list:
            # Adding to data list
            data_list.append(data_dict)
            print(f'Link {counter} added')
    else:
        print(counter)
        print('Skipped due to missing info.')

df = pd.DataFrame(data_list)
df.head()

""


In [27]:
country = 'India'
csv_filename = f"{country}_First_Post.csv"

# Checking if the CSV file already exists
if os.path.exists(csv_filename):
    existing_df = pd.read_csv(csv_filename)
    # Merging new and existing dataframe
    df = pd.concat([existing_df, pd.DataFrame(data_list)], ignore_index=True)
    df["date"] = pd.to_datetime(df["date"])  # Converting the "date" column to datetime
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.sort_values(by="date", ascending=False)  # Sorting the date
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)  
else:
    # If csv file does not exist, then we create a new CSV file with the scraped data
    df = pd.DataFrame(data_list)
    df["date"] = pd.to_datetime(df["date"], format = "%d-%m-%Y")  # Converting the "date" column to datetime
    df = df.sort_values(by="date", ascending=False)
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)